In [55]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

# from torchsummary import summary

import matplotlib.pyplot as plt
from PIL import Image

import pathlib

import time

In [52]:
data_paths = {
    'train': "content/cassavaleafdata/train",
    'valid': "content/cassavaleafdata/validation", 
    'test': "content/cassavaleafdata/test"
}

def loader(path):
    return Image.open(path)

img_size = 150

train_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(
    root=str(data_paths['train']),
    loader=loader,
    transform=train_transforms
)
valid_data = datasets.ImageFolder(
    root=str(data_paths['valid']),
    transform=train_transforms
)
test_data = datasets.ImageFolder(
    root=str(data_paths['test']),
    transform=train_transforms
)

In [53]:
batch_size = 256

train_dataloader = data.DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True
)
test_dataloader = data.DataLoader(
    dataset=test_data,
    batch_size=batch_size
)
valid_dataloader = data.DataLoader(
    dataset=valid_data,
    batch_size=batch_size
)

In [54]:
for inputs, labels in train_dataloader:
    print(inputs.shape)
    break

torch.Size([256, 3, 150, 150])


In [56]:
device = torch.device('gpu' if torch.cuda.is_available() else 'cpu')

# LeNet CNN architecture for classification
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        # Initialize parent class
        super().__init__()
        # First conv layer: 1 input channel (grayscale), 6 output channels, 5x5 kernel with same padding
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=6, kernel_size=5, padding='same'
        )
        # First pooling layer: 2x2 average pooling
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        # Second conv layer: 6 input channels, 16 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(
            in_channels=6, out_channels=16, kernel_size=5
        )
        # Second pooling layer: 2x2 average pooling
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        # Flatten layer to convert 2D feature maps to 1D vector
        self.flatten = nn.Flatten()
    
        self.fc_1 = nn.Linear(16*35*35, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        # inputs shape: (batch_size, 1, 150, 150)
        
        # Pass through first conv layer
        # outputs shape: (batch_size, 6, 150, 150) - same padding preserves dimensions
        outputs = self.conv1(inputs)
        
        # Apply first average pooling
        # outputs shape: (batch_size, 6, 75, 75) - halved spatial dimensions
        outputs = self.avgpool1(outputs)
        
        # Apply ReLU activation - shape remains (batch_size, 6, 75, 75)
        outputs = F.relu(outputs)
        
        # Pass through second conv layer
        # outputs shape: (batch_size, 16, 71, 71) - no padding reduces spatial dims by 4
        outputs = self.conv2(outputs)
        
        # Apply second average pooling
        # outputs shape: (batch_size, 16, 35, 35) - halved spatial dimensions
        outputs = self.avgpool2(outputs)
        
        # Apply ReLU activation - shape remains (batch_size, 16, 35, 35)
        outputs = F.relu(outputs)
        
        # Flatten 2D feature maps to 1D
        # outputs shape: (batch_size, 16*35*35)
        outputs = self.flatten(outputs)
        # Pass through first FC layer
        # outputs shape: (batch_size, 120)
        outputs = self.fc_1(outputs)
        # Pass through second FC layer
        # outputs shape: (batch_size, 84)
        outputs = self.fc_2(outputs)
        # Pass through output FC layer
        # outputs shape: (batch_size, num_classes)
        outputs = self.fc_3(outputs)
        return outputs

In [57]:
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    # Create DataLoader if not already a DataLoader
    if not isinstance(train_dataloader, torch.utils.data.DataLoader):
        train_dataloader = torch.utils.data.DataLoader(
            train_dataloader,
            batch_size=32,
            shuffle=True
        )

    for idx, (inputs, labels) in enumerate(train_dataloader):
        # Move batch to device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(inputs)

        # Calculate loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        # Calculate accuracy
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        
        # Print progress
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(train_dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss


def evaluate(model, criterion, valid_dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []

    # Create DataLoader if not already a DataLoader
    if not isinstance(valid_dataloader, torch.utils.data.DataLoader):
        valid_dataloader = torch.utils.data.DataLoader(
            valid_dataloader,
            batch_size=32,
            shuffle=False
        )

    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_dataloader):
            # Move batch to device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            predictions = model(inputs)

            # Calculate loss
            loss = criterion(predictions, labels)
            losses.append(loss.item())

            # Calculate accuracy
            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [58]:
num_classes = len(train_data.classes)

lenet_model = LeNetClassifier(num_classes=num_classes)
lenet_model.to(device=device)

criterion = torch.nn.CrossEntropyLoss()
learning_rate = 2e-4
optimizer = optim.Adam(lenet_model.parameters(), learning_rate)

num_epochs = 10
save_model = './model'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100

for epoch in range(1, num_epochs+1):
    epoch_start_time = time.time()

    train_acc, train_loss = train(
        model=lenet_model,
        optimizer=optimizer,
        criterion=criterion,
        train_dataloader=train_data,
        device=device,
        epoch=epoch
    )

    eval_acc, eval_loss = evaluate(
        model=lenet_model,
        criterion=criterion,
        valid_dataloader=valid_data
    )
    eval_losses.append(eval_loss)

    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model.pt')

    print("-" * 59)
    print(
        "| End of epoch {:3d} | Time: {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} "
        
        "| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss
        )
    )
    print("-" * 59)

    lenet_model.load_state_dict(torch.load(save_model + "/lenet_model.pt"))
    lenet_model.eval()

| epoch   1 |    50/  177 batches | accuracy    0.463
| epoch   1 |   100/  177 batches | accuracy    0.481
| epoch   1 |   150/  177 batches | accuracy    0.489
-----------------------------------------------------------
| End of epoch   1 | Time: 227.49s | Train Accuracy    0.501 | Train Loss    1.314 | Valid Accuracy    0.508 | Valid Loss    1.268 
-----------------------------------------------------------
| epoch   2 |    50/  177 batches | accuracy    0.538
| epoch   2 |   100/  177 batches | accuracy    0.539
| epoch   2 |   150/  177 batches | accuracy    0.541
-----------------------------------------------------------
| End of epoch   2 | Time: 229.49s | Train Accuracy    0.566 | Train Loss    1.202 | Valid Accuracy    0.546 | Valid Loss    1.233 
-----------------------------------------------------------
| epoch   3 |    50/  177 batches | accuracy    0.563
| epoch   3 |   100/  177 batches | accuracy    0.580
| epoch   3 |   150/  177 batches | accuracy    0.576
----------

In [59]:
test_dataloader = data.DataLoader(
    test_data,
    batch_size=batch_size
)
test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader)
test_acc, test_loss

(0.5824933687002652, 1.3226159699261189)